In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
import pyspark.sql

In [2]:
spark:SparkSession= SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/26 08:25:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [30]:
c = spark.read.option("multiLine", "true").csv('../sample_data/channel.csv',header=True)
cl = spark.read.option("multiLine", "true").csv('../sample_data/channel_log.csv',header=True)

In [31]:
channel = cl.join(c[['name','topic','channel_id']],on='channel_id',how='left')

In [32]:
channel = channel.dropna().sort('created_date',ascending=False).drop_duplicates(subset=['channel_id'])

In [33]:
topic_list = [item['topic'] for item in channel.select('topic').collect()]
topic_list = [item.replace(' ','').split(',') for item in topic_list]
topic_list = [item for row in topic_list for item in row]

In [34]:
topic_list[:5]

['Entertainment',
 'Lifestyle_(sociology)',
 'Action-adventure_game',
 'Role-playing_video_game',
 'Action_game']

In [35]:
from collections import Counter

In [36]:
c = Counter(topic_list)
top_topics = [item[0] for item in c.most_common()[:7]]
print(top_topics)
for idx,item in enumerate(top_topics):
    if item == 'Lifestyle_(sociology)':
        top_topics[idx] = 'Lifestyle'
    elif 'Music' in item:
        top_topics[idx] = 'Music'
print(top_topics)

['Lifestyle_(sociology)', 'Entertainment', 'Food', 'Music', 'Film', 'Music_of_Asia', 'Society']
['Lifestyle', 'Entertainment', 'Food', 'Music', 'Film', 'Music', 'Society']


In [37]:
def filter_category(x):
    for topic in top_topics:
        if topic in x:
            return topic
    return 'Other'
udf_filter_category = udf(filter_category)
# result = list(map(check_category,[item['topic'] for item in channel.select('topic').collect()]))

In [38]:
channel = channel.withColumn('topic',udf_filter_category('topic'))

In [48]:
pd_channel = channel.toPandas()
pd_channel = pd_channel.astype(
    {'view_count': int,
    'sub_count': int,
    'video_count':int
    }
)

In [52]:
import plotly.express as px

fig = px.scatter(pd_channel, x="video_count", y="sub_count",color='topic',
                 log_x=True,
                 size='view_count',
                 labels={
                     "sub_count": "Num of Subscriber",
                     "video_count": "Num of video (log)",
                     "name": "Channel Name",
                     "topic": "Categories",
                     "view_count":"Num of views"
                 },
                 hover_name='name',
                 hover_data={
                    "topic":True,
                    "sub_count":True,
                    "video_count":True,
                    "video_count":False,
                },
                 category_orders={
                              "topic": [*top_topics,"Other"]},
                 title="Top youtube channels in TW",
                #  autosize=False,
                 width=800,
                 height=500,                
                 template="plotly_dark"
                 )
fig.show()

/Users/Harry/Documents/GitHub/top_youtuber_dashboard_tw/.venv/lib/python3.11/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [27]:
# fig.write_image('../images/channel_distribution.png')
# fig.write_html('../images/channel_distribution.html')